In [ ]:
# 1. Install necessary Python libraries
!pip install -q streamlit opencv-python-headless tensorflow numpy pandas pillow

# 2. Install Localtunnel to create the public URL
!npm install -q -g localtunnel

⠙⠹⠸⠼⠴⠦⠧
changed 22 packages in 975ms
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [ ]:
!streamlit cache clear

In [ ]:
%%writefile app.py
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
import streamlit as st
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd
import io
from datetime import datetime
from PIL import Image
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input

# --- FIX FOR POTENTIAL MODEL LOADING ISSUES ---
class PatchedInputLayer(InputLayer):
    def __init__(self, *args, **kwargs):
        if 'batch_shape' in kwargs:
            kwargs['batch_input_shape'] = kwargs.pop('batch_shape')
        super().__init__(*args, **kwargs)

tf.keras.utils.get_custom_objects()['InputLayer'] = PatchedInputLayer

# --- APP CONFIGURATION ---
MODEL_PATH = "efficientnetb0_facial_skin_final.h5"
FACE_CASCADE_PATH = "haarcascade_frontalface_default.xml"
CLASS_NAMES = ["Clear Skin", "Dark Spots", "Puffy Eyes", "Wrinkles"]

@st.cache_resource
def load_assets():
    model = load_model(MODEL_PATH, compile=False)
    face_cascade = cv2.CascadeClassifier(FACE_CASCADE_PATH)
    return model, face_cascade

# --- PAGE SETUP ---
st.set_page_config(page_title="AI Facial Skin Analyzer", layout="wide")

# --- SIDEBAR & HISTORY ---
st.sidebar.title("Settings & History")
mode = st.sidebar.radio("Choose Input Mode", ["Upload Photo", "Live Camera"])

if 'history' not in st.session_state:
    st.session_state.history = []

st.title("🧠 AI Facial Skin Analyzer")

# --- LOAD MODEL ---
if not os.path.exists(MODEL_PATH) or not os.path.exists(FACE_CASCADE_PATH):
    st.error("⚠️ Files missing! Please upload the model and XML files to Colab.")
else:
    model, face_cascade = load_assets()

    img_file = None
    if mode == "Upload Photo":
        img_file = st.file_uploader("Upload Image", type=["jpg", "png", "jpeg"])
    else:
        img_file = st.camera_input("Take a snapshot for analysis")

    if img_file:
        # Optimization: Immediate resizing for speed
        image = Image.open(img_file).convert("RGB")

        # Downscale image to max width of 800px to reduce processing load
        max_width = 800
        if image.width > max_width:
            ratio = max_width / float(image.width)
            new_height = int(float(image.height) * float(ratio))
            image = image.resize((max_width, new_height), Image.Resampling.LANCZOS)

        img_array = np.array(image)
        original_img = img_array.copy()
        display_img = img_array.copy()
        gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)

        # Faster face detection parameters
        faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(50, 50)
        )

        results_log = []

        # Process multiple faces
        for i, (x, y, w, h) in enumerate(faces):
            face_roi = img_array[y:y+h, x:x+w]

            # Use INTER_AREA for faster/cleaner downsampling
            face_roi_resized = cv2.resize(face_roi, (224, 224), interpolation=cv2.INTER_AREA)

            face_roi_preprocessed = preprocess_input(face_roi_resized)
            face_roi_batched = np.expand_dims(face_roi_preprocessed, axis=0)

            # Inference
            preds = model.predict(face_roi_batched, verbose=0)[0]
            idx = np.argmax(preds)
            label = CLASS_NAMES[idx]
            prob = preds[idx]

            # Drawing annotations
            cv2.rectangle(display_img, (x, y), (x+w, y+h), (0, 255, 0), 4)
            cv2.putText(display_img, f"#{i+1}: {label} ({prob:.1%})", (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            results_log.append({
                "Face_ID": i + 1,
                "Timestamp": datetime.now().strftime("%H:%M:%S"),
                "Condition": label,
                "Confidence": f"{prob:.2%}"
            })

        # Display results
        col1, col2 = st.columns(2)
        with col1:
            st.subheader("Captured/Uploaded")
            st.image(original_img, use_container_width=True)
        with col2:
            st.subheader("AI Analysis")
            st.image(display_img, use_container_width=True)

        if results_log:
            df_results = pd.DataFrame(results_log)
            st.divider()
            st.subheader("📋 Detection Details")
            st.table(df_results)

            st.session_state.history.extend(results_log)

            # Download Actions
            st.markdown("### 📥 Download Results")
            dl_col1, dl_col2 = st.columns(2)

            with dl_col1:
                csv = df_results.to_csv(index=False).encode('utf-8')
                st.download_button(
                    label="Download Report (CSV)",
                    data=csv,
                    file_name=f"analysis_{datetime.now().strftime('%H%M%S')}.csv",
                    mime='text/csv',
                )

            with dl_col2:
                result_img_pil = Image.fromarray(display_img)
                buf = io.BytesIO()
                result_img_pil.save(buf, format="JPEG", optimize=True, quality=85)
                st.download_button(
                    label="Download Annotated Image",
                    data=buf.getvalue(),
                    file_name=f"analyzed_{datetime.now().strftime('%H%M%S')}.jpg",
                    mime="image/jpeg",
                )
        else:
            st.warning("No faces detected. Ensure your face is clearly visible.")

# Sidebar History Display
if st.session_state.history:
    st.sidebar.markdown("---")
    st.sidebar.write("Recent Activity:")
    st.sidebar.dataframe(pd.DataFrame(st.session_state.history).tail(10))
    if st.sidebar.button("Clear History"):
        st.session_state.history = []
        st.rerun()

Overwriting app.py


In [ ]:
import urllib
# 1. Get the Password for the tunnel
print("STEP 1: Copy this Tunnel Password/IP:")
print(urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip())
print("\nSTEP 2: Click the link below (ending in .loca.lt) to open the UI in a new tab.")

# 2. Start Streamlit and Localtunnel
!streamlit run app.py & npx localtunnel --port 8501

STEP 1: Copy this Tunnel Password/IP:
34.73.19.130

STEP 2: Click the link below (ending in .loca.lt) to open the UI in a new tab.
⠙

⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼your url is: https://violet-clouds-float.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.73.19.130:8501

2026-01-02 07:12:04.962865: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-02 07:12:04.968108: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-02 07:12:04.983401: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767337925.009708   14242 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for 